In [1]:
# 导入数据
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
# 定义超参数
num_epochs = 20
batch_size = 64
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
# 构建pipeline，对图像处理
pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [4]:
# 下载、加载数据集
train_set = dsets.MNIST(root='./data', train=True, download=True, transform=pipeline) 
test_set = dsets.MNIST(root='./data', train=True, download=True, transform=pipeline) 

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5) #1：灰度图片，10：输出通道，5：krenel
        self.conv2 = nn.Conv2d(10, 20, 3) #10：输入通道，20：输出通道，3：krenel
        self.fc1 = nn.Linear(20*10*10, 500) # 20*10*10：输入通道， 500：输出通道
        self.fc2 = nn.Linear(500, 10) #500：输入通道，10：输出通道
    
    def forward(self, x):
        input_size = x.size(0) #batch_size * 1 * 28 * 28
#         print(x.shape)
        x = self.conv1(x) #输入：batch_size*1*28*28，输出：batch_size*10*24*24 (24=28-5+1)
        x = F.relu(x) #保持shape不变，输出：batch_size*10*24*24
        x = F.max_pool2d(x, 2, 2) #输入：batch_size*10*24*24 输出：batch_size*10*12*12
#         print(x.shape)
        
        x = self.conv2(x) #输入：batch_size*10*12*12，输出：batch_size*20*10*10
        x = F.relu(x) #保持shape不变
        
        x = x.view(input_size, -1) #拉平， -1 自动计算维度， 20*20*10=2000
        
        x = self.fc1(x) #输入：batch*2000，输出：batch*500
        x = F.relu(x) #保持shape不变
        
        x = self.fc2(x) #输入：batch*500，输出：batch*10
        
        output = F.log_softmax(x, dim=1) #dim=1 按行计算
        
        return output

In [6]:
# 定义优化器
model = ConvNet().to(device)
# criterion
optimizer = optim.Adam(model.parameters())

In [7]:
# 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 部署到device上去
        data, target = data.to(device), target.to(device)
        # 梯度初始化为0
        optimizer.zero_grad()
        # 训练后的结果
        output = model(data)
        # 计算损失
        loss = F.cross_entropy(output, target)
        # 找到概率值最大的下标
#         pred = output.max(1, keepdim=True) # pred = output.argmax(dim=1)
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
        if batch_idx % 3000 == 0:
            print('Train Epoch:{}\t Loss:{:.6f}'.format(epoch, loss.item()))

In [8]:
# 定义测试方法
def test_model(model, device, test_loader):
    # 模型验证
    model.eval()
    # 正确率
    correct = 0.0
    # 测试损失
    test_loss = 0.0
    with torch.no_grad():
        for data, target in test_loader:
            # 部署到device上去
            data, target = data.to(device), target.to(device)  
            # 测试数据
            output = model(data)
            # 计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            # 找到概率值最大的下标
            pred = output.max(1, keepdim=True)[1] # 值，索引
            # pred = output.argmax(dim=1)
            # pred = torch.max(output, dim=1)
            # 累计正确率
        correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print('Test Average Loss:{:.4f}, Accuracy:{:.3f}\n'.format(
            test_loss, 100.0 * correct / len(test_loader.dataset)))

In [9]:
# 调用方法
for epoch in range(1, num_epochs+1):
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

Train Epoch:1	 Loss:2.323325
Test Average Loss:0.0008, Accuracy:0.053

Train Epoch:2	 Loss:0.016252
Test Average Loss:0.0006, Accuracy:0.052

Train Epoch:3	 Loss:0.014927
Test Average Loss:0.0003, Accuracy:0.053

Train Epoch:4	 Loss:0.015072
Test Average Loss:0.0003, Accuracy:0.053

Train Epoch:5	 Loss:0.021786
Test Average Loss:0.0001, Accuracy:0.053

Train Epoch:6	 Loss:0.025607
Test Average Loss:0.0002, Accuracy:0.053

Train Epoch:7	 Loss:0.000970
Test Average Loss:0.0001, Accuracy:0.053

Train Epoch:8	 Loss:0.000427
Test Average Loss:0.0002, Accuracy:0.053

Train Epoch:9	 Loss:0.004821
Test Average Loss:0.0001, Accuracy:0.053

Train Epoch:10	 Loss:0.013559
Test Average Loss:0.0001, Accuracy:0.053

Train Epoch:11	 Loss:0.000425
Test Average Loss:0.0001, Accuracy:0.053

Train Epoch:12	 Loss:0.001959
Test Average Loss:0.0000, Accuracy:0.053

Train Epoch:13	 Loss:0.000078
Test Average Loss:0.0001, Accuracy:0.053

Train Epoch:14	 Loss:0.000017
Test Average Loss:0.0001, Accuracy:0.053

T